# Books corpus topic model

In [1]:
import pandas as pd
import numpy as np

friends = pd.read_csv('../transcripts and metadata/friends_mastersheet.csv', index_col=0)
friends.head()

,Year_of_prod,Season,Episode Number,Episode_Title,Duration,Summary,Director,Stars,Votes,match,...,Phoebe,Ross,Rachel,Carol,Susan,Janice,Mike,Gunther,Ben,Emily
0,1994,1,1,The One Where Monica Gets a Roommate: The Pilot,22,"Monica and the gang introduce Rachel to the ""r...",James Burrows,8.3,7440,11,...,0.061248,0.144114,0.155643,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,1994,1,2,The One with the Sonogram at the End,22,Ross finds out his ex-wife is pregnant. Rachel...,James Burrows,8.1,4888,12,...,0.061500,0.248342,0.165269,0.088068,0.044696,0.000000,0.0,0.0,0.0,0.0
2,1994,1,3,The One with the Thumb,22,Monica becomes irritated when everyone likes h...,James Burrows,8.2,4605,13,...,0.145415,0.125624,0.102383,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,1994,1,4,The One with George Stephanopoulos,22,Joey and Chandler take Ross to a hockey game t...,James Burrows,8.1,4468,14,...,0.111161,0.151849,0.162042,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,1994,1,5,The One with the East German Laundry Detergent,22,"Eager to spend time with Rachel, Ross pretends...",Pamela Fryman,8.5,4438,15,...,0.095007,0.150061,0.141764,0.000000,0.000000,0.047483,0.0,0.0,0.0,0.0


In [2]:
np.mean(friends['len_prep'])

686.228813559322

## Some checks

In [3]:
# 5 least popular Friends episodes

check = friends.sort_values(by='Stars')

for i in range(5):
    row = check.iloc[i]
    episode = row['Episode Number']
    name = row['Episode_Title']
    season = row['Season']
    summary = row['Summary']
    stars = row['Stars']
    print(f'season {season} episode {episode} , "{name}" rating = {stars} stars: \n\n {summary}\n')

season 4 episode 21 , "The One with the Invitation" rating = 7.2 stars: 

 As Ross and Emily send out wedding invitations, he and Rachel each look back on their relationship together while Joey and Chandler look for a purpose in life.

season 6 episode 20 , "The One with Mac and C.H.E.E.S.E." rating = 7.4 stars: 

 Chandler forgets to write down a message for Joey when the time of his audition for the lead in a TV show is changed.

season 8 episode 19 , "The One with Joey's Interview" rating = 7.5 stars: 

 Joey prepares for his epic interview with Soap Opera Digest, reminiscing about his past acting experiences, his friends and his romantic life.

season 7 episode 21 , "The One with the Vows" rating = 7.5 stars: 

 Monica and Chandler are getting married in four weeks and have to write their vows, but neither has a clue what to say, so they reflect on their past.

season 9 episode 10 , "The One with Christmas in Tulsa" rating = 7.5 stars: 

 Chandler must spend Christmas in Tulsa, whi

In [4]:
# 5 most popular Friends episodes

check = friends.sort_values(by='Stars', ascending=False)

for i in range(5):
    row = check.iloc[i]
    episode = row['Episode Number']
    name = row['Episode_Title']
    season = row['Season']
    summary = row['Summary']
    stars = row['Stars']
    print(f'season {season} episode {episode} , "{name}" rating = {stars} stars: \n\n {summary}\n')

season 10 episode 18 , "The Last One: Part 2" rating = 9.7 stars: 

 Phoebe races Ross to the airport in a bid to stop Rachel from leaving for Paris. Monica and Chandler pack up their apartment ahead of their move to their new house, and Joey buys Chandler a new chick and duck as a leaving present.

season 5 episode 14 , "The One Where Everybody Finds Out" rating = 9.7 stars: 

 When Phoebe finds out about Monica and Chandler, she and Rachel attempt to get them to confess their relationship.

season 10 episode 17 , "The Last One: Part 1" rating = 9.5 stars: 

 Erica gives birth to the baby that Monica and Chandler are adopting. However, there's one small added surprise. Meanwhile, Ross and Rachel sleep together one last time before Rachel leaves ...                See full summary »

season 4 episode 12 , "The One with the Embryos" rating = 9.5 stars: 

 Phoebe's uterus is examined for implantation of the embryos. Meanwhile, a seemingly harmless game between Chandler and Joey against M

## prep

In [5]:
import glob
import random
RND = 12345
random.seed(RND)

class Chunker(object):
    def __init__(self, filenames, metadata, singles_only=False,
                 max_chars_per_book=None, chunk_size=None):
        
        # maximum number of characters to read from the beginning of each book:
        self.max_chars_per_book = max_chars_per_book
        
        # size of the chunks returned (in tokens):
        self.chunk_size = chunk_size
        
        # only return chunks from these files:
        self.filenames = filenames
        
        # include a dictionary with metadata:
        self.meta = meta
        
        # only return texts with a single genre label:
        self.singles_only = singles_only
        
        # only collect genres during the first iteration
        self.first = True

    def __iter__(self):
        if self.first:
            self.genres = []
        
        for filename in self.filenames:
            title = os.path.basename(filename)
            current_genres = self.meta[title]
            
            if self.singles_only:
                if len(current_genres) > 1:
                    continue

            with open(filename, 'r', encoding='utf-8') as f:
                try:
                    if self.max_chars_per_book:
                        text = f.read()
                        text = text[:self.max_chars_per_book]
                    else:
                        text = f.read()
                except:
                    continue
            
            tokens = text.split()
            
            for i in range(0, len(tokens), self.chunk_size):
                if self.first:
                    self.genres.append(current_genres)
                yield tokens[i:i + self.chunk_size]
                #this also helps for lazy loading because yield goes over it apart
        
        # stop collecting labels after first complete iteration
        self.first = False

In [6]:
n_features = 5000 # number of most common words
n_topics = 30 # number of topics
n_top_words = 60 # number of characteristic words per topic (viz purposes only)
max_df = 0.5 # maximum document frequency
min_df = 100 # minimum document frequency
max_books = 1000 # include all books (None), or a specific number
chunk_size = 686  # length of the average friends episode (after keeping only ADJ, NOUN & VERB)

In [7]:
path = '../../session-09-20220505T113620Z-001/session-09/hp_topic_model/books/books/*.txt'
filenames = sorted(list(glob.glob(path)))
print(len(filenames))

6800


In [8]:
random.seed(RND)
random.shuffle(filenames)
# print(filenames[:10])
# print(len(filenames))

In [9]:
import json
with open('../../session-09-20220505T113620Z-001/session-09/hp_topic_model/books_meta.json', 'r') as f:
    meta = json.loads(f.read())

# for book, genres in list(meta.items())[:20]:
#     print(f"{book} -> {' - '.join(sorted(genres))}")

## NMF

In [10]:
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import joblib

def identity(x):
    return x

chunks = Chunker(filenames, meta,
                 chunk_size=chunk_size,
                 max_chars_per_book=None)


# vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df,
#                              max_features=n_features,
#                              analyzer=identity)

vectorizer = joblib.load('books_vectorizer.sav')

X = vectorizer.transform(chunks)

from sklearn.decomposition import NMF
# nmf = NMF(n_components=300,
#           random_state=RND,
#           verbose=1, max_iter=50).fit(X) # fit NMF model with TF-IDF feature
nmf = joblib.load('nmf_books_friends.sav')
# joblib.dump(vectorizer, 'books_vectorizer.sav')

In [11]:
joblib.dump(vectorizer.vocabulary_, 'books_vectorizer_vocab.sav')
vectorizer.get_feature_names_out()
joblib.dump(vectorizer.get_feature_names_out(), 'books_features_names.sav')

['books_features_names.sav']

In [12]:
# import joblib
# joblib.dump(nmf, 'nmf_books_friends.sav')

In [13]:
for topic_idx, topic in enumerate(nmf.components_[:10]):
    top_idxs = np.argsort(topic)[::-1][:10]
    top_words = np.array(vectorizer.get_feature_names())[top_idxs]
    print(f'topic {topic_idx}:', ' - '.join(top_words))

topic 0: question - answer - ask - questions - answered - asking - answers - feeling - thinking - wondering
topic 1: air - sound - fear - breath - scream - throat - tried - panic - hear - lungs
topic 2: mouth - tongue - taste - teeth - throat - sucking - lip - opened - sucked - bite
topic 3: says - asks - looks - ask - nods - smiles - turns - replies - tells - adds
topic 4: father - died - study - proud - anger - born - taught - childhood - angry - learned
topic 5: mom - okay - honey - hug - sweetie - crazy - boyfriend - weird - driveway - cancer
topic 6: hell - damn - bastard - ass - damned - bitch - crap - pissed - fact - goddamn
topic 7: lot - big - nice - bit - getting - stuff - bad - liked - thinking - having
topic 8: ship - ships - vessel - cargo - fleet - port - space - hatch - board - engines
topic 9: men - moved - armed - faces - seen - guns - dozen - wounded - their - carried


c:\Users\Gebruiker\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
nmf.verbose = 1
X_ = nmf.transform(X)
print(X_.shape)

violation: 1.0
violation: 0.18986899875577293
violation: 0.10465565379056027
violation: 0.017839851677198607
violation: 0.005571748429768093
violation: 0.002139511926052226
violation: 0.0003835171593940432
violation: 0.00020764634272646458
violation: 5.472903630991739e-05
Converged at iteration 10
(252493, 300)


In [17]:
from sklearn.feature_selection import f_classif

genres = [g[0] for g in chunks.genres]
feature_names = vectorizer.get_feature_names()

for genre in set(genres):
    print(f'-> genre {genre}:')
    y = [1 if g == genre else 0 for g in genres]
    # univariate feature selection with F-test for feature scoring
    F, pval = f_classif(X_, y)
    
    # indices of three best topics
    for topic_idx in np.argsort(F)[::-1][:3]:
        topic = np.nan_to_num(nmf.components_[topic_idx])
        words = [feature_names[i] for i in topic.argsort()[:-10:-1]]
        print('   ' + f't{topic_idx} ' ' - '.join(words))

c:\Users\Gebruiker\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


-> genre Thriller:
   governmentt177  - countryt177  - militaryt177  - americant177  - securityt177  - politicalt177  - publict177  - presidentt177  - news
   bondt125  - telephonet125  - mt125  - yardst125  - cardst125  - bigt125  - greyt125  - machinet125  - centre
   courtt86  - trialt86  - lawyert86  - juryt86  - judget86  - lawt86  - lawyerst86  - clientt86  - case
-> genre Themes:
   futuret290  - knowledget290  - differentt290  - historyt290  - persont290  - naturet290  - selft290  - pointt290  - experience
   gazet239  - glancedt239  - staredt239  - expressiont239  - smilet239  - steppedt239  - liftedt239  - narrowedt239  - murmured
   oldt68  - yeart68  - aget68  - fashionedt68  - ancientt68  - oldert68  - youngert68  - wrinkledt68  - dusty
-> genre Mystery:
   caset46  - informationt46  - murdert46  - killert46  - evidencet46  - investigationt46  - crimet46  - filet46  - agent
   policet253  - stationt253  - officerst253  - copst253  - detectivet253  - murdert253  - chieft253

In [33]:
import pandas as pd
nmf.verbose = 0

texts = [value.split() for value in friends.text.values]

scores = nmf.transform(vectorizer.transform(texts))  # transform the HP chapters according to the fitted NMF model
scores.shape
# print(scores)

(236, 300)

In [34]:
df_scores = pd.DataFrame(scores, columns=[f't{i}' for i in range(scores.shape[1])])
friends2 = friends.join(df_scores)
friends2.head()

,Year_of_prod,Season,Episode Number,Episode_Title,Duration,Summary,Director,Stars,Votes,match,...,t290,t291,t292,t293,t294,t295,t296,t297,t298,t299
0,1994,1,1,The One Where Monica Gets a Roommate: The Pilot,22,"Monica and the gang introduce Rachel to the ""r...",James Burrows,8.3,7440,11,...,0.000000,0.000000,0.003622,0.0,0.000000,0.000008,0.000726,0.0,0.008491,0.000010
1,1994,1,2,The One with the Sonogram at the End,22,Ross finds out his ex-wife is pregnant. Rachel...,James Burrows,8.1,4888,12,...,0.000869,0.000286,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.003331,0.000057
2,1994,1,3,The One with the Thumb,22,Monica becomes irritated when everyone likes h...,James Burrows,8.2,4605,13,...,0.003109,0.000000,0.000000,0.0,0.000000,0.000221,0.005802,0.0,0.000000,0.000000
3,1994,1,4,The One with George Stephanopoulos,22,Joey and Chandler take Ross to a hockey game t...,James Burrows,8.1,4468,14,...,0.000000,0.000058,0.000000,0.0,0.000000,0.000351,0.000000,0.0,0.021268,0.000386
4,1994,1,5,The One with the East German Laundry Detergent,22,"Eager to spend time with Rachel, Ross pretends...",Pamela Fryman,8.5,4438,15,...,0.000000,0.000694,0.000000,0.0,0.000055,0.000000,0.002872,0.0,0.007433,0.000830


In [35]:
friends2.to_csv('friends_books_topics_model.csv')